## 목표 : 발화문에 대한 Summarization 성능 높이기

youtube 스크립트 요약 프로젝트를 위해 SKT koBART, 그 중 gogamza님의 summarization 모델을 불러와 사용하였ek. 그러나 발화문 및 짧은 문장에 대한 요약성능이 좋지않아 자체 데이터로 fine tuning을 시도하게 되었다.

<br>

파인튜닝 데이터셋으로 AI Hub의 한국어 대화요약 데이터셋, 방송 스크립트 요약 데이터셋, Github, Hugging Face에 존재하는 데이터셋을 확보해 비교하였고, 그 중 가장 테스크에 적합한 길이와 일상문을 포함하는 Hugging Face의 koconversation dataset을 활용하기로 결정했다.

<br>

데이터를 살펴본 결과 원본 대화문이 카톡 데이터로 이루어진 채팅 데이터셋에 가까웠고, 이를 전처리하여 발화문 형식으로 고쳐 활용하는 방안을 사용하였다.

In [1]:
!git clone https://github.com/seujung/KoBART-summarization.git

Cloning into 'KoBART-summarization'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 151 (delta 48), reused 41 (delta 39), pack-reused 83
Receiving objects: 100% (151/151), 37.24 MiB | 31.49 MiB/s, done.
Resolving deltas: 100% (78/78), done.


In [5]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/conv_train_.tsv',delimiter='\t')
df = df[['news', 'summary']]

In [7]:
df

,news,summary
0,월욜날 면접임 면접은 언제 오 나도 통햇네 헐 그케 빨리? 대박이네 자기소개 같은거...,월요일(월욜)이 면접이라고 하니까 유튜브에 괜찮은 자기소개 영상 등이 많을 테니 찾...
1,이렇게 해도 야근할거 같아..... 진짜 할거많은가보다 오늘 행사준비까지 갔으면 큰...,오늘은 야근해야 할 것 같고 내일은 행사 준비를 가야 한다.
2,근데 주휴수당이 유급휴가를 준다는말이잖아 뭔지 잘 이해안돼... 한번 찾아볼께에 웅...,주당 15시간 이상 근무한 사람에게 무조건 주휴수당을 줘야 한다.
3,근데 휴가 끝나야 현장 시작할 듯 휴가 못 바꿈. 지시 내려왔으 8월1일~10일 ...,지시가 내려와서 휴가를 바꿀 수가 없다.
4,#@이름#야 나 낼 자젼거 못탈듯!!! 이번주에 면접이 좌좌좌작 생겻어!!!! 헐키...,이번 주에 면접이 줄줄이 생겨서 내일 자전거를 못 타게 되었다.
...,...,...
279987,나 왜캐 다른운동 하기싫냐 싸이클만해맨날 팔뚣 해야되는데 진짜 개하기싫어 싸이클이...,매일 사이클만 하고 팔뚝 운동을 해야 하는데 다른 운동은 하기 싫은 사람이 러닝머신...
279988,이틀째면 배많이아플텐데 따듯한곳에서 쉬어용.. 약을 먹어서 괜찮아졋어용 껄껄 근육통...,이틀째인데 아래쪽이 너무 당겨서 진통제와 근육통 약을 같이 먹으니 좋아졌다.
279989,웅 그럼 5두개 6한개 산화제 3프로 이케 사면 되는 거지 나 지금.엄청 어둡자나 ...,"뿌리 염색(뿌염)을 하기 싫어서 컬러 다운할때 쓰는 3프로의 산화제와 5 두 개, ..."
279990,근데 낼 근데 스 그 앞머리 저르케 하는 거 파마 하까 앞머리 조금망? 얍 고데기 ...,탈색 모는 녹기 때문에 앞머리 파마을 할 수가 없다.


<br>

데이터 셋은 크게 두가지의 특징을 가졌다. 첫번째로, 채팅에서만 사용되는 한국어 음절 이모티콘이 활용되었으며, 그 외의 이모티콘 또는 개인정보는 모두 일괄적인 패턴하에 마스킹되었다.  



이를 대처하기 위해, 대표적으로 감정을 나타내는 이모티콘인 'ㅠㅠ', 'ㅋㅋ' 등을 정규표현식으로 일괄 삭제하였고, '#@시스템#동영상#', '#@이름#'과 같이 마스킹된 데이터들은 삭제하거나 "철수", "민수" 와 같이 이를 일괄적으로 대체할만한 대체텍스트를 각 패턴마다 알맞게 삽입하였다.

전체적인 프로세스는 분업 작업으로 이루어져, 현재 코드상으로는 정규표현식으로 이모티콘 삭제, 마스킹 패턴 탐색과정만 존재하고 있고, 이를 tsv 형식으로 교환하며 작업하였고, 최종 전처리를 마친 데이터셋은 conv_train_.tsv, conv_test_.tsv으로 만들어져 실제 파인튜닝을 진행하였다.

In [10]:
import re

# 마스킹 패턴 "#@패턴1#" 탐색 함수
def find_patterns1(text):
  return re.findall(r'#@\w+#', text)


# 마스킹 패턴 "#@패턴1#", "#@패턴1#패턴2#" 탐색 함수
# "#@패턴1#패턴2#", "#@패턴1# ... #@패턴1#" 을 구분하기 위해, "#패턴2" 가 발생되는 "시스템", "이모티콘" 에 대해서만 탐색을 진행하였다.
def find_patterns2(text):
  result = []
  for i in re.findall(r'#@\w+#\w+#',text) :
    if i[:6] == "#@시스템#" or i[:7] == "#@이모티콘#" :
      result.append(i)
    else :
      i = re.findall(r'#@\w+#', i)
      result.extend(i)

  result += re.findall(r'#@\w+#', text)
  return result

# 예시
conversations = [
    '지금은 애들이 일찍자니까. 전에 수업끝나고 애들보고간다고. #@이름# 고모 는 왜그러세요. 집도 가까운디. 왜 야밤에 ㅋㅋ. 요즘은 자기수업가기전에 애들보고싶다고. 데리고 오라함. #@이모티콘# 집으로. 세상에.',
    '#@시스템#동영상#. 으앙 넘 귀여워. #@시스템#동영상#. 미쳤다ㅠㅠㅠㅠㅠㅠㅠ. ㅠㅠ졸귀탱이지ㅠㅠㅜㅠ. 고개가 획 돌아간다 ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅍ. 귀엽짘ㅋㅋㅋㅋㅋ. 쟤 잘때 귀에 간식이라고 속삭이면 일어난다. #@이모티콘#ㅋㅋㅋ귀여워ㅠㅠㅠ. 나중에 찍어볼겤ㅋㅋㅋㅋㅋㅋㅋ.',
]

for conv in conversations:
    print(find_patterns2(conv))

['#@이름#', '#@이모티콘#']
['#@시스템#동영상#', '#@시스템#동영상#', '#@시스템#', '#@시스템#', '#@이모티콘#']


In [62]:
# 마스킹 패턴 탐색

masking_set = set()
result = []
for i in range(len(df)) :
  masking = find_patterns2(df.iloc[i,0])
  masking_set.update(masking)

for mask in masking_set :
  print(mask)

#@이모티콘#당황#
#@이모티콘#절규#
#@이모티콘#방긋#
#@이모티콘#선물#
#@이모티콘#아픔#
#@이모티콘#삐짐#
#@이모티콘#졸려#
#@이모티콘#수줍#
#@이모티콘#민망#
#@이모티콘#
#@기타#
#@이름#
#@이모티콘#쳇#
#@이모티콘#놀람#
#@이모티콘#눈물#
#@이모티콘#소주#
#@시스템#검색#
#@시스템#
#@전번#
#@이모티콘#악마#
#@이모티콘#정색#
#@이모티콘#축하#
#@이모티콘#케익#
#@이모티콘#크크#
#@이모티콘#미소#
#@이모티콘#야호#
#@이모티콘#음표#
#@이모티콘#곤란#
#@이모티콘#풍선껌#
#@이모티콘#땀#
#@이모티콘#딸기#
#@이모티콘#좌절#
#@이모티콘#부끄#
#@이모티콘#행복#
#@이모티콘#짜증#
#@이모티콘#신나#
#@이모티콘#최악#
#@이모티콘#브이#
#@이모티콘#발그레#
#@이모티콘#해#
#@이모티콘#맥주#
#@이모티콘#꺄아#
#@시스템#삭제#
#@이모티콘#최고#
#@이모티콘#훌쩍#
#@이모티콘#컴온#
#@이모티콘#와인#
#@소속#
#@이모티콘#썩소#
#@이모티콘#쑥스#
#@이모티콘#총#
#@이모티콘#하트뿅#
#@이모티콘#윙크#
#@이모티콘#헤어나오고싶지않은것#
#@이모티콘#헉#
#@이모티콘#찡긋#
#@이모티콘#하트#
#@이모티콘#별#
#@이모티콘#잠#
#@이모티콘#멘붕#
#@이모티콘#그만#
#@이모티콘#우웩#
#@이모티콘#제발#
#@이모티콘#잘난척#
#@시스템#지도#
#@이모티콘#오케이#
#@이모티콘#뿌듯#
#@이모티콘#컵케익a#
#@번호#
#@시스템#동영상#
#@이모티콘#힘듦#
#@이모티콘#열받아#
#@이모티콘#메롱#
#@이모티콘#근심#
#@이모티콘#감동#
#@이모티콘#뽀뽀#
#@이모티콘#우와#
#@이모티콘#심각#
#@이모티콘#깜찍#
#@이모티콘#깜짝#
#@이모티콘#부르르#
#@이모티콘#안도#
#@이모티콘#카톡#
#@이모티콘#씨익#
#@이모티콘#커피#
#@이모티콘#흡족#
#@이모티콘#으으#
#@시스템#파일#
#@이모티콘#흑흑#
#@이모티콘#치킨#
#@이모티콘#담배#
#@URL#
#@계정#
#@주소#
#@이모티콘

In [2]:
import re

# 초성 이모지 삭제 함수
def remove_single_consonants_and_vowels(text):
    return re.sub(r'[ㄱ-ㅎㅏ-ㅣ]', '', text)

conversations = [
    '지금은 애들이 일찍자니까. 전에 수업끝나고 애들보고간다고. 고모는 왜그러세요. 집도 가까운디. 왜 야밤에 ㅋㅋ. 요즘은 자기수업가기전에 애들보고싶다고. 데리고 오라함. 집으로. 세상에.',
    '#@시스템#동영상#. 으앙 넘 귀여워. #@시스템#동영상#. 미쳤다ㅠㅠㅠㅠㅠㅠㅠ. ㅠㅠ졸귀탱이지ㅠㅠㅜㅠ. 고개가 획 돌아간다 ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅍ. 귀엽짘ㅋㅋㅋㅋㅋ. 쟤 잘때 귀에 간식이라고 속삭이면 일어난다. ㅋㅋㅋ귀여워ㅠㅠㅠ. 나중에 찍어볼겤ㅋㅋㅋㅋㅋㅋㅋ.',
    '심심하군. 힙합 리액션 보여줘. ㅇㄷ. 비행기. 곧 내림. 마중 나왔지?. ㅇㅇㅇ. 기달. ㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱ. 이미 버스인데?. 엥. 나 기다리는 중인데????. 다시 와.',
    '아엄마웃기다. 아빠한텐왜말했어ㅡㅡ. 진짜고자질쟁이ㅡㅡ. 입1g. 짜증나 ㅡㅡ. 내기분 생각해주는건. 아빠뿐. 뭐래. 나도 들어줬자나. 근데 엄마가 뭐라구 안함 ?. ㅇㅇ그냥 별말 안하더랑. 지금은 화 많이 풀렸나봥 ㅎㅎ.',
    '운전하는 사람도 힘들지만. 안 운전하는 사람도 힘들엌ㅋㅋㅋ. ㅋㅋㅋㅋㅋㅋ. 심지어 조수석. 자라는데도 잘 못자겟고 ㅋㅋㅋㅋㅋㅋ. 막판에 버티다 버티다 항복 ㅋㅋㅋㅋㅋ. 어우 조수석은 부담스러워ㅋㅋㅋㅋㅋㅋㅋㅋㅋ. 쫑알쫑알 해야되니까ㅜ. ㅋㅋㅋㅋㅋㅋㅋ. 마죠.'
]

for conv in conversations:
    print(remove_single_consonants_and_vowels(conv))

지금은 애들이 일찍자니까. 전에 수업끝나고 애들보고간다고. 고모는 왜그러세요. 집도 가까운디. 왜 야밤에 . 요즘은 자기수업가기전에 애들보고싶다고. 데리고 오라함. 집으로. 세상에.
#@시스템#동영상#. 으앙 넘 귀여워. #@시스템#동영상#. 미쳤다. 졸귀탱이지. 고개가 획 돌아간다 . 귀엽짘. 쟤 잘때 귀에 간식이라고 속삭이면 일어난다. 귀여워. 나중에 찍어볼겤.
심심하군. 힙합 리액션 보여줘. . 비행기. 곧 내림. 마중 나왔지?. . 기달. . 이미 버스인데?. 엥. 나 기다리는 중인데????. 다시 와.
아엄마웃기다. 아빠한텐왜말했어. 진짜고자질쟁이. 입1g. 짜증나 . 내기분 생각해주는건. 아빠뿐. 뭐래. 나도 들어줬자나. 근데 엄마가 뭐라구 안함 ?. 그냥 별말 안하더랑. 지금은 화 많이 풀렸나봥 .
운전하는 사람도 힘들지만. 안 운전하는 사람도 힘들엌. . 심지어 조수석. 자라는데도 잘 못자겟고 . 막판에 버티다 버티다 항복 . 어우 조수석은 부담스러워. 쫑알쫑알 해야되니까. . 마죠.


In [64]:
for i in range(len(df)) :
  df.iloc[i,0] = remove_single_consonants_and_vowels(df.iloc[i,0])

In [65]:
df

,news,summary
0,월욜날 면접임 면접은 언제 오 나도 통햇네 헐 그케 빨리? 대박이네 자기소개 같은거...,월요일(월욜)이 면접이라고 하니까 유튜브에 괜찮은 자기소개 영상 등이 많을 테니 찾...
1,이렇게 해도 야근할거 같아..... 진짜 할거많은가보다 오늘 행사준비까지 갔으면 큰...,오늘은 야근해야 할 것 같고 내일은 행사 준비를 가야 한다.
2,근데 주휴수당이 유급휴가를 준다는말이잖아 뭔지 잘 이해안돼... 한번 찾아볼께에 웅...,주당 15시간 이상 근무한 사람에게 무조건 주휴수당을 줘야 한다.
3,근데 휴가 끝나야 현장 시작할 듯 휴가 못 바꿈. 지시 내려왔으 8월1일~10일 ...,지시가 내려와서 휴가를 바꿀 수가 없다.
4,#@이름#야 나 낼 자젼거 못탈듯!!! 이번주에 면접이 좌좌좌작 생겻어!!!! 헐키...,이번 주에 면접이 줄줄이 생겨서 내일 자전거를 못 타게 되었다.
...,...,...
279987,나 왜캐 다른운동 하기싫냐 싸이클만해맨날 팔뚣 해야되는데 진짜 개하기싫어 싸이클이...,매일 사이클만 하고 팔뚝 운동을 해야 하는데 다른 운동은 하기 싫은 사람이 러닝머신...
279988,이틀째면 배많이아플텐데 따듯한곳에서 쉬어용.. 약을 먹어서 괜찮아졋어용 껄껄 근육통...,이틀째인데 아래쪽이 너무 당겨서 진통제와 근육통 약을 같이 먹으니 좋아졌다.
279989,웅 그럼 5두개 6한개 산화제 3프로 이케 사면 되는 거지 나 지금.엄청 어둡자나 ...,"뿌리 염색(뿌염)을 하기 싫어서 컬러 다운할때 쓰는 3프로의 산화제와 5 두 개, ..."
279990,근데 낼 근데 스 그 앞머리 저르케 하는 거 파마 하까 앞머리 조금망? 얍 고데기 ...,탈색 모는 녹기 때문에 앞머리 파마을 할 수가 없다.


실제로는 이모지 삭제 함수 및 패턴정보를 넘겨주고 다른 컴퓨터에서 대체 텍스트 삽입 및 패턴삭제를 진행하였음

In [ ]:
df

In [ ]:
df.isna().sum()

news       0
summary    0
dtype: int64

In [ ]:
df.dropna(axis=1)

,news,summary
0,호로요이 아직도 안파나 호로요이는 근데 맛 너무 많아서 아 그거말고 매실 맛나는 약...,호로요이와 매화수의 술맛에 대해 토론한다.
1,나 지금 배민에 새로운 가게 봤거든? 웅웅 시키려고? 아니 평점이 1점인거야? 그래...,배달의 민족(배민)에 있는 신규 가게 리뷰 중 수저를 안 줬다는 이유로 별점을 1개...
2,오늘은 아이스크림 먹었나? 응 먹었다 무슨맛으로 먹었는데 그 흑임자 아이스크림 있잖...,오늘 편의점에서 파는 흑임자 아이스크림을 먹었다.
3,#@이름# 오늘 쉬는 날 ㅎㅎ 히히 진짜? 좋겠당 하루종일 모해용 나 카페 갈 준비...,오늘 쉬는 날이어서 동네 카페에 가려고 한다.
4,아니 내가 과일을 싫어하는건 아냐 그럼?? 왜안머겅 먹ㄱㅣ 귀찮을 뿐ㅇㅣ야 방울 토...,과일을 싫어하는 것이 아니라 먹을 때 귀찮은 것이라 후식으로 설빙에서 과일빙수 먹을...
...,...,...
34999,이대로자면 아쉬워서 우짜꼬 우야꼬 진짜 우야꼬 ㅋㅋㅋㅋㅋㅋ 하필이면 지금 1화를 보...,지금 1화를 봐서 이대로 자면 아쉽지만 일단은 자기로 한다.
35000,스타트업 다 봤어? 난 이제 10화 보는데 말야 남주 바껴? 안 바뀔듯 근데 드라마...,스타트업 10화 보고 있는데 남자 주인공(남주) 별로여서 수지 얼굴이랑 옷 입은 것...
35001,지금 도안 먹고 제작 중 퀘 끝내거 15분 만에 얻음 킥킥 머야 금방 구했네? 예리...,퀘스트(퀘) 끝내고 15분 만에 도안 먹고 제작 중인데 반지 강화하다 강화 레벨(렙...
35002,그리고 너무 코르셋이 심해 엘사가 왕인데 오프숄더가 말이되냐 추워디지겠구만 내복같은...,영화 겨울왕국의 설정이 과하다고 주장하고 있다.


## koBART Fine Tuning

In [ ]:
!pip install -r /content/KoBART-summarization/requirements.txt


ERROR: Could not find a version that satisfies the requirement torch==1.10.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.10.0


In [ ]:
!pip install pandas
# torch==1.10.0
!pip install transformers==4.8.2
!pip install pytorch-lightning==1.3.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 20.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895239 sha256=22dddb95c7947e1c164b2ae18699745ab01a85468777c4241c7fdc1c7910c45a
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully buil

In [ ]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 15.2 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.7 MB/s eta 0:00:00


In [ ]:
import os
import tarfile

target_folder = '/content/KoBART-summarization/data/train.tar.gz'
with tarfile.open(target_folder, 'r:gz') as f:
    f.extractall()

import pandas as pd

df = pd.read_csv('/content/train.tsv')
print(df)

In [ ]:
import os
os.chdir('/content/KoBART-summarization')

In [ ]:
!pwd

/content/KoBART-summarization


In [ ]:
!python train.py  --gradient_clip_val 1.0  \
                 --max_epochs 50 \
                 --default_root_dir logs \
                 --gpus 1 \
                 --batch_size 4 \
                 --num_workers 4

2023-07-31 06:20:56.385599: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/KoBART-summarization/train.py", line 160, in <module>
    parser = pl.Trainer.add_argparse_args(parser)
AttributeError: type object 'Trainer' has no attribute 'add_argparse_args'


In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, PreTrainedTokenizerFast
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, tokenizer, source_texts, target_texts, max_length=512):
        self.tokenizer = tokenizer
        self.source_texts = source_texts
        self.target_texts = target_texts
        self.max_length = max_length

    def __len__(self):
        return len(self.source_texts)

    def __getitem__(self, index):
        source_text = str(self.source_texts[index])
        target_text = str(self.target_texts[index])

        # Tokenize the source and target texts
        source_tokens = self.tokenizer.batch_encode_plus([source_text], max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        target_tokens = self.tokenizer.batch_encode_plus([target_text], max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'input_ids': source_tokens['input_ids'].squeeze(),
            'attention_mask': source_tokens['attention_mask'].squeeze(),
            'labels': target_tokens['input_ids'].squeeze(),
        }

# Example training data
source_texts = ["Input text 1", "Input text 2", ...]
target_texts = ["Target text 1", "Target text 2", ...]

# Initialize the BART tokenizer and model
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

# Create the dataset and dataloader
dataset = CustomDataset(tokenizer, source_texts, target_texts)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Set device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(dataloader)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss}")

# Save the trained model
model.save_pretrained("trained_bart_model")
tokenizer.save_pretrained("trained_bart_model")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


KeyboardInterrupt: ignored